In [1]:
from dlisio import dlis
from dataclasses import dataclass
import pathlib
import pandas as pd
from pandas import DataFrame
from lisbelo.searcher import Searcher
from lisbelo.mnemonicfix import MnemonicFix
from lisbelo.json import JSON_Transfer


In [2]:
root = pathlib.Path('/Users/giba/Pedro_Work/MONUMENTO/_MASTER_APPS/LisBelo/pocos')
dlis_list = Searcher.SearchDlis(root)

In [24]:
@dataclass
class DlisBelo:

    MergedLogicalFiles: list[dlis.LogicalFile]

    def __init__(self, dlisfile: dlis.PhysicalFile,path) -> None:
        self.path = path
        self.MergedLogicalFiles = []
        self.MergedFrames = []
        self.DataDictList = []

        self.MergeFiles(dlisfile)
        self.GetFrames()
        self.GetOrigins()
        self.MakeDataDict()

    def MergeFiles(self, dlisfile: dlis.PhysicalFile) -> None:
        """
        Junta os arquivos lógicos
        """
        *files, = dlisfile
        self.MergedLogicalFiles.extend(files)

    def GetFrames(self, ):
        """
        Junta os frames
        """
        self.origins = []
        for file in self.MergedLogicalFiles:
            for origin in file.origins:

                self.origins.extend(origin.file_id)

            self.MergedFrames.extend(file.find('FRAME'))

    def GetOrigins(self):
        self.origins = []
        for file in self.MergedLogicalFiles:
            for origin in file.origins:

                self.origins.append(origin.file_id)


    def MakeDataDict(self):
        for frames in self.MergedFrames:
            self.DataDict(frames)

    def DataDict(self, frame:dlis.Frame):

        DataDict = {}
        DataDict['WELL'] = self.origins[0]
        DataDict['FRAME'] = frame.name
        DataDict['CURVES'] = [ch.name for ch in frame.channels]
        DataDict['UNIT'] = [ch.units for ch in frame.channels]

        df = pd.DataFrame(frame.curves(strict=False))
        df = MnemonicFix.IndexToDept(df)
        df = MnemonicFix.gammarename(df)
        
        df['WELL'] = self.origins[0]
        DataDict['DATAFRAME'] = df
        self.DataDictList.append(DataDict)


In [ ]:
@dataclass
class DataDict:
    
    def __init__(self,frame:dlis.Frame) -> None:
        pass

In [25]:
class LisPlaza:
    
    DlisFileList:list[DlisBelo]

    def __init__(self,pathlist:list[pathlib.Path]) -> None:
        self.DlisFileList=[]
        self.OpenList(pathlist)

    def OpenList(self,pathlib:list[pathlib.Path]) -> None:
        """
        Se encarrega de abrir e fechar o arquivo
        """
        for path in pathlib:
            # - Abrindo o arquivo
            dl = dlis.load(f'{path}')
            # - Classe custom
            dlbelo = DlisBelo(dl,path)
            # - Aicionando na lista
            self.DlisFileList.append(dlbelo)
            # - Fechando
            dl.close()

    def GammaScan(self):
        """
        Procura em todos os arquivos GR
        """
        GammaData = []
        for file in self.DlisFileList:
            for dataframe in file.DataDictList:
                if 'GR' in dataframe['DATAFRAME'].columns.values:
                    GammaData.append(dataframe)
        
        return GammaData

    def FileNames(self):
        """
        Essa função retorna 
        """
        for file in self.DlisFileList:
            print(file.DataDictList['WELL'])

        

In [26]:
Dlis_Plaza = LisPlaza(dlis_list)

In [34]:
poco_teste = Dlis_Plaza.DlisFileList[2]

In [27]:
gamma1 = Dlis_Plaza.GammaScan()
#len(gamma1)

In [42]:
for file in  poco_teste.MergedLogicalFiles:
    print(file.tools)


[Tool(ES)]
[Tool(ES)]
[Tool(ES)]


In [48]:
for file in Dlis_Plaza.DlisFileList:
    for logical in file.MergedLogicalFiles:
        print(f"{logical.tools} pertence ao arquivo {file.DataDictList[0]['WELL']}")

[Tool(IES)] pertence ao arquivo 1PO  0001  MABRA1PO  0001  MA1
[Tool(FDC/GR)] pertence ao arquivo 1PO  0001  MABRA1PO  0001  MA1
[Tool(BHC)] pertence ao arquivo 1PO  0001  MABRA1PO  0001  MA1
[Tool(IES)] pertence ao arquivo 1PO  0001  MABRA1PO  0001  MA1
[Tool(BHC)] pertence ao arquivo 1PO  0001  MABRA1PO  0001  MA1
[Tool(FDC/GR)] pertence ao arquivo 1PO  0001  MABRA1PO  0001  MA1
[Tool(CST_GR)] pertence ao arquivo 1PO  0001  MABRA1PO  0001  MA1
[Tool(WST)] pertence ao arquivo 1PO  0001  MABRA1PO  0001  MA1
[Tool(ES)] pertence ao arquivo 1MO  0001  MABRA1MO  0001  MA1
[Tool(ES)] pertence ao arquivo 1MO  0001  MABRA1MO  0001  MA1
[Tool(ML)] pertence ao arquivo 1MO  0001  MABRA1MO  0001  MA1
[Tool(ES)] pertence ao arquivo 1MO  0001  MABRA1MO  0001  MA1
[Tool(ES)] pertence ao arquivo 1MO  0001  MABRA1MO  0001  MA1
[Tool(ML/ML)] pertence ao arquivo 1MO  0001  MABRA1MO  0001  MA1
[Tool(ES)] pertence ao arquivo 1MO  0001  MABRA1MO  0001  MA1
[Tool(ES)] pertence ao arquivo 1MO  0001  MABRA1MO

In [30]:
nome_teste = gamma1[10]['WELL']
nome_teste.split()

['1RPR', '0001', 'MABRA1RPR', '0001', 'MA1']

In [20]:
import matplotlib.pyplot as plt

class GammaPlot:

    @staticmethod
    def singleGamma(df_: DataFrame, title: str = None, save=False):

        fig,ax = plt.subplots(figsize=(10, 15), dpi=300)      
        ax1 = plt.subplot2grid((1, 1), (0, 0), rowspan=1, colspan=1)
        fig.suptitle(title, fontsize=26)

        ylim = (df_['DEPT'].max(), df_['DEPT'].min())
        ax1.plot('GR', 'DEPT', data=df_)  # Call the data from the df dataframe
        ax1.set_xlabel("Gamma")  # Assign a track title
        ax1.set_xlim(0, 200)  # Change the limits for the curve being plotted
        ax1.set_ylim(ylim)  # Set the depth range
        ax1.grid()  # Display the grid
        if save:
            plt.savefig(f'{title}.png')
            plt.close(fig)
            return


In [21]:
for df in gamma1:
    GammaPlot.singleGamma(df_=df['DATAFRAME'],title=(f"{df['WELL']}"),save=True)

{'WELL': '2PR  0001  MABRA2PR  0001  MA1',
 'NAME': '[0,1,ISF/BHC/GR/BOREHOLE-DEPTH/1/.50/F]',
 'CURVES': ['INDEX4114',
  'DT_1',
  'GR',
  'GR_1',
  'ILD',
  'ILD_1',
  'SFLA',
  'SFLA_1',
  'SFLU',
  'SFLU_1',
  'SP',
  'SP_1',
  'DT'],
 'UNIT': ['ft',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 'DATAFRAME':        FRAMENO    DEPT    DT_1          GR    GR_1     ILD   ILD_1    SFLA  \
 0            1    76.0 -999.25   26.855469 -999.25 -999.25 -999.25 -999.25   
 1            2    76.5 -999.25   26.496094 -999.25 -999.25 -999.25 -999.25   
 2            3    77.0 -999.25   26.210938 -999.25 -999.25 -999.25 -999.25   
 3            4    77.5 -999.25   26.140625 -999.25 -999.25 -999.25 -999.25   
 4            5    78.0 -999.25   25.750000 -999.25 -999.25 -999.25 -999.25   
 ...        ...     ...     ...         ...     ...     ...     ...     ...   
 14352    14353  7252.0 -999.25 -999.250000 -999.25 -999.25 -999.25 -999.25   
 14